In [12]:
# Let's do our usual data science imports
import pandas as pd
import numpy as np
import matplotlib as plot
import sklearn as sk
import importlib
import os
import subprocess


# Define a number of constants
STANDARD_MUTATION = 0.5
STANDARD_MOMENTUM = 0
STANDARD_POPSIZE = 100
STANDARD_SIGMA = 0.004
PATH_TO_LOG = "storage/logs/"
    
EVALUATION_SCHAFFERS = "SchaffersEvaluation"
EVALUATION_KATSUURA = "KatsuuraEvaluation"
EVALUATION_BCF = "BentCigarFunction"
EVALUATION_SPHERE = "SphereEvaluation"

PARAMETER_POPSIZE = "popsize"
PARAMETER_MOMENTUM = "Momentum"
PARAMETER_MUTATION = "Mutation"
PARAMETER_SIGMA = "Sigma"

In [5]:
# Define the Setup class
class Setup:
    """A setup for a defined parameter and evaluation, with given parameter values.

    Params:
        - param_name (str): Parameter to define
        - evaluation (str): Name of evaluation function to optimise.
        - parameter_values (list): List of values for the associated parameter
    """
    def __init__(self, param_name = "", evaluation = "", parameter_values = []):
        self.param_name = param_name
        self.evaluation = evaluation
        self.parameter_values = parameter_values
        self.parameter_scores = []

In [27]:
# Define the Simulator class
class Simulator:
    """A simulator for a givenn number of setups, and runs to test on each parameter value of a setup.

    Params:
        - setups(list): A list of Setup to test on
        - runs(int): Number of runs per setup
    """
    
    def __init__(self, setups, runs=10):
        self.setups = setups
        self.runs = runs

    def run(self):
        for setup in self.setups:
            self.runEvalsForSetup(setup)

    def constructParams(self, setup, param):
        """Sets up default and preconfigured parameters."""
        mutation = STANDARD_MUTATION
        momentum = STANDARD_MOMENTUM
        sigma = STANDARD_SIGMA
        popsize = STANDARD_POPSIZE

        if setup.param_name == PARAMETER_MUTATION:
            mutation = param

        if setup.param_name == PARAMETER_MOMENTUM:
            momentum = param

        if setup.param_name == PARAMETER_SIGMA:
            sigma = param
        
        if setup.param_name == PARAMETER_POPSIZE:
            popsize = param

        return {
            'mutation': mutation,
            'momentum': momentum,
            'sigma': sigma,
            'popsize': popsize
        }

    def runEvalsForSetup(self, setup):
        """Runs for the current Setup:setup of parameters to test. """
        for param in setup.parameter_values:
            params = self.constructParams(setup, param)
            self.runEval(setup, params, param)

    def runEval(self, setup, params, param):
        """Runs the EA on the igiven eval"""
        scores = 0
        print(scores)
        
        for i in range(self.runs):
            cmd = "bash run.sh %s %d %f" % (setup.evaluation, params['popsize'], params['sigma'])
            p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            output = p.stdout.readlines()
            self.storeDataToFiles(output, setup, params, param, i)
            
    def storeDataToFiles(self, output, setup, params, param, run_nr):
        path = PATH_TO_LOG + f"{setup.evaluation}/{setup.param_name}/{setup.param_name}-val-{param}-run-{run_nr}.txt"
        
        if not os.path.exists(os.path.dirname(path)):
            try:
                os.makedirs(os.path.dirname(path))
            except OSError as exc: # Guard against race condition
                if exc.errno != errno.EEXIST:
                    raise

        file = open(path, "w+")
        file.write(str(output))
        file.close()

In [28]:
# Initialize a number of setups
katsuura_popsize_setup = Setup(
    PARAMETER_POPSIZE, 
    EVALUATION_KATSUURA,
    np.linspace(50, 500, 50)
)

schaffers_popsize_setup = Setup(
    PARAMETER_POPSIZE, 
    EVALUATION_SCHAFFERS,
    [50, 100]
#     np.linspace(50, 500, 50)
)

setups = [schaffers_popsize_setup]
mainSimulator = Simulator(setups)

In [29]:
mainSimulator.run()

0
0
